In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from scipy.sparse import *
from collections import defaultdict
from itertools import *
from sklearn.metrics import pairwise_distances
import tqdm

In [2]:
conf = SparkConf().setAppName("App")
conf.set("spark.driver.memory", '20g')
#conf = (conf.setMaster('local[*]')
        #.set('spark.driver.memory', '4G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))
#FOR THE TOP POP
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [4]:
print(trainSet.count())
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
print("userItem shape:",userItem.shape)
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))
print("itemFeat shape:",itemFeature.shape)

170149
userItem shape: (15375, 37143)
itemFeat shape: (37143, 19716)


In [5]:
numberOfRecommendations=5
#TOP POPULAR
cost=8
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict = defaultdict(int)
seenItemsDict = seenItems.collectAsMap()
#--------------------------------------------------------------

In [6]:
userFeature = userItem.dot(itemFeature)
userFeature.shape

(15375, 19716)

In [7]:
data = []
rows = []
cols = []
for f in featureIdfDict.keys():
    data.append(featureIdfDict[f])
    cols.append(f)
    rows.append(f)
featureIdf = csr_matrix((data,(rows,cols)))
featureIdf.shape        

(19716, 19716)

In [8]:
userProfile = userFeature.dot(featureIdf)
userProfile

<15375x19716 sparse matrix of type '<class 'numpy.float64'>'
	with 653062 stored elements in Compressed Sparse Row format>

In [9]:
userSimilarities = 1 - pairwise_distances(userProfile, metric='cosine')

In [10]:
type(userSimilarities)

numpy.ndarray

In [11]:
np.fill_diagonal(userSimilarities,0)

In [12]:
k_value = 50
userList = trainSet.map(lambda x: x[0]).distinct().collect()
numUser = len(userList)

In [13]:
values, rows, cols = [], [], []
for user in userList:
    currentUserSimilarity = userSimilarities[user][:]
    top_k_idx = np.argsort(currentUserSimilarity)[-k_value:]
    values.extend(currentUserSimilarity[top_k_idx])
    rows.extend(np.ones(k_value) * user)
    cols.extend(top_k_idx)

In [14]:
similarity_topK = csr_matrix((values, (rows, cols)), dtype=np.float32)

In [15]:
similarity_topK = similarity_topK + similarity_topK.transpose()

In [16]:
print("Sim top k shape: ", similarity_topK.shape)
print("userItem: ", userItem.shape)
print("ItemUser: ", userItem.transpose().shape)

Sim top k shape:  (15375, 15375)
userItem:  (15375, 37143)
ItemUser:  (37143, 15375)


In [17]:
predRatings = userItem.transpose()
print(type(userItem.transpose()))
print(type(similarity_topK.transpose()))

<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>


In [18]:
predRatings = userItem.transpose().dot(similarity_topK).transpose()

In [19]:
predRatings.shape

(15375, 37143)

In [20]:
its = np.argsort(predRatings[4][:].toarray()[0])[-10:][::-1]
print(its)
print("Ratings: ", predRatings[4][:].toarray()[0][its])

[25656 23217 32578 17380 28102 32117 18741 15378 31731 34271]
Ratings:  [ 113.12700625   50.49530581   28.56198211   11.07715186   10.83322167
    9.91448776    9.37620401    8.47393513    8.4468416     8.08896518]


In [24]:
#Time to generate the predictions!
f=open("predictionsCollViaContentNoSeen.csv",'w')
numberOfRecommendations = 5
f.write("userId,RecommendedItemIds\n")
i=0
for user in targets:
    currentItems = predRatings[user][:].toarray()[0]
    reccomended = np.argsort(currentItems)[-numberOfRecommendations:][::-1]
    #unseenMask = np.in1d(reccomended, seenItemsDict.get(user,[]), invert = True)
   # reccomended = reccomended[unseenMask][-numberOfRecommendations:][::-1]
    print(reccomended)
    f.write(str(user)+',')
    for prod in reccomended:
        f.write(str(prod)+' ')
    f.write('\n')
    i = i + 1
    print(i,"of", len(targets), "written")
f.close()

[25656 23217 32578 17380 28102]
1 of 4196 written
[17815  2164 22211 15743 33755]
2 of 4196 written
[13473 25421 20407 28102 36853]
3 of 4196 written
[ 3674 31444 30287 21867 34390]
4 of 4196 written
[ 9855 15469  5861 23137 22870]
5 of 4196 written
[17782  1448 32019 18741 17107]
6 of 4196 written
[11366 26454 19188 19036 28905]
7 of 4196 written
[36204 33562 24488  4735  6821]
8 of 4196 written
[ 9607 13069 23833 26332 22976]
9 of 4196 written
[14322 29261 33562 27686 21613]
10 of 4196 written
[ 9123 19798 18803 19748 10963]
11 of 4196 written
[30653 29308 19022 28634  9735]
12 of 4196 written
[19370  3665  9893 14387  6189]
13 of 4196 written
[19987 12866 26585  5366  2911]
14 of 4196 written
[33282 26346 32083 22776 24488]
15 of 4196 written
[ 8216 33266 20941 17207 15973]
16 of 4196 written
[11012 34713 28294 35738 11557]
17 of 4196 written
[10701 36872 35300 23282  4708]
18 of 4196 written
[ 9535 34715  7393 33092 31836]
19 of 4196 written
[11717  3680 13782  9476 28964]
20 of 41

[ 7903 28701 18383 35381 31136]
202 of 4196 written
[35381 33546 10514  2464 31992]
203 of 4196 written
[28964 11366 25304 15971  4807]
204 of 4196 written
[ 1076 15743 33500 17648 26285]
205 of 4196 written
[ 1397 37065 21495 26621  2623]
206 of 4196 written
[ 2232 19725 35186  7923 36537]
207 of 4196 written
[21613  1456 17472 35390 25421]
208 of 4196 written
[21484 32578 14572  4735 17782]
209 of 4196 written
[28568  2339 35390  3077 14414]
210 of 4196 written
[34715 12669 21336  4967  7393]
211 of 4196 written
[ 7368 34715 31963 21336 14076]
212 of 4196 written
[ 9577  9074 16367 33443 21912]
213 of 4196 written
[36872 30898 18200  1076  7597]
214 of 4196 written
[33649  1053 31506   133  1363]
215 of 4196 written
[33266  8216 20941 23317 15973]
216 of 4196 written
[10469 12362   269 14480  6111]
217 of 4196 written
[37132 22925 13470 31076 30087]
218 of 4196 written
[25304 10645 33755  1203 33562]
219 of 4196 written
[36872  6612 20495 28199 25467]
220 of 4196 written
[   52 19001

[ 2339 24880  8549 14572  3077]
399 of 4196 written
[34715  8013 23843  9913 14076]
400 of 4196 written
[ 7393 27660  2911 25694 19752]
401 of 4196 written
[27132 22330 21919 31836  4731]
402 of 4196 written
[ 6461  9913 36957  1363 32291]
403 of 4196 written
[15971 34595 14878 14020 10059]
404 of 4196 written
[ 8013 34715 23843  6454 14076]
405 of 4196 written
[ 7659  7933 21840  7824  5237]
406 of 4196 written
[11083 34129  1834 35549 24563]
407 of 4196 written
[23701  2031 28102 27583 28200]
408 of 4196 written
[17645  5524 26024   876  7287]
409 of 4196 written
[ 8599 12222 28063 12019 16656]
410 of 4196 written
[ 9052  2453 13649 26951 27132]
411 of 4196 written
[34715 14076  4967 22338  9913]
412 of 4196 written
[ 6258  5747 20518 22029 33500]
413 of 4196 written
[ 1289 23951 29219  8447 20413]
414 of 4196 written
[26621 11458 37065 29046 14616]
415 of 4196 written
[21913 34930   228 26332  7923]
416 of 4196 written
[10701  4708 23282 18987  7984]
417 of 4196 written
[21765 37065

598 of 4196 written
[29452 29807 33500 21157 26502]
599 of 4196 written
[ 5447 25366 24377 29184  8063]
600 of 4196 written
[19426 20620 23336    88 11707]
601 of 4196 written
[35129 26884 33155  9893 27464]
602 of 4196 written
[ 9657 30799 14767  8659 16575]
603 of 4196 written
[11012  1076 28294 17648 18200]
604 of 4196 written
[17856 36749  3451  8846 16367]
605 of 4196 written
[30360 14738  4554 14616 16818]
606 of 4196 written
[15971 14872 24639 28701 12997]
607 of 4196 written
[  269  6263 35738  6446 25304]
608 of 4196 written
[17529 30925 25421 33562 17611]
609 of 4196 written
[32347 28950  8189  7948 15930]
610 of 4196 written
[ 9420 33562 29221 30595 24639]
611 of 4196 written
[34799 16656 17973 26468  3704]
612 of 4196 written
[11259 26621  2561 29046 21495]
613 of 4196 written
[ 6699 33562 35391  6821  7923]
614 of 4196 written
[15570 26681  4753 31836 34949]
615 of 4196 written
[  876 15687 26024 35158 27780]
616 of 4196 written
[33341 14322 30104  9902 29184]
617 of 4196 

[26594 28083 37065  1397 23044]
795 of 4196 written
[ 2339 26804 25304 27451  1289]
796 of 4196 written
[29736 13451 29570 27562 33603]
797 of 4196 written
[25920 19127 22976  8216 33266]
798 of 4196 written
[33598 15508  9534  7592  7811]
799 of 4196 written
[  269 22002 19798 35738 14372]
800 of 4196 written
[26454 36942 17240 25696  8442]
801 of 4196 written
[25363  4105 15593  6343 16571]
802 of 4196 written
[18996 20489  5297 28547 34172]
803 of 4196 written
[30727 12271 21644 10031 13000]
804 of 4196 written
[25257 15126 11617 32797 20262]
805 of 4196 written
[ 7342 22963 16876 22458  8205]
806 of 4196 written
[30904 35905 16339 34446  7555]
807 of 4196 written
[17815 35381 13367  1076 10514]
808 of 4196 written
[31076  7150 18562 25925 13502]
809 of 4196 written
[ 1730  5905 29323 35065 16694]
810 of 4196 written
[ 3457 19127 21765 26951 29815]
811 of 4196 written
[18178  8549 35738  3040 25624]
812 of 4196 written
[19478 22272  2227 27455 16272]
813 of 4196 written
[32019 26951

...

[32915 36434  3496   146 32038]
3376 of 4196 written
[31978 14780  5993 24760 10672]
3377 of 4196 written
[ 7923 21707 25925  5834 35100]
3378 of 4196 written
[15928  8049 20489  8108 30668]
3379 of 4196 written
[17815  1076 15743 17648  2164]
3380 of 4196 written
[26491   269 35738 34426 18139]
3381 of 4196 written
[24342 17149 25656 16916 24479]
3382 of 4196 written
[32347 32025 16367 19241 35318]
3383 of 4196 written
[30360 14738  4554 14616 16818]
3384 of 4196 written
[21717 33298 21081 24547 18546]
3385 of 4196 written
[11266 23260 16153 13323 23239]
3386 of 4196 written
[ 5128 28688 23030 31362 33383]
3387 of 4196 written
[ 6461  9913 34715 12669 14076]
3388 of 4196 written
[22762 20356 25150 21406 27419]
3389 of 4196 written
[12997  6627 17690 24639  8268]
3390 of 4196 written
[  775 10548 32293 12293 10427]
3391 of 4196 written
[34351  4833 29570 22029 20283]
3392 of 4196 written
[18200 33475 17383  2712  2108]
3393 of 4196 written
[  418 29301 19426 26836  4386]
3394 of 4196 w

[ 8426 34403 11081 24164 33663]
3574 of 4196 written
[20941  8216 15973 33266 15917]
3575 of 4196 written
[24345 23030 13625 30592 18105]
3576 of 4196 written
[14636 11774 14572 25656 25939]
3577 of 4196 written
[11259 27569 30360 19748 16826]
3578 of 4196 written
[36869 27080  5227 19529 17747]
3579 of 4196 written
[27441 34156  1209 35945 22628]
3580 of 4196 written
[30514 30704 19130 34070 27544]
3581 of 4196 written
[33818 18063 13220 27753 17011]
3582 of 4196 written
[ 4708  7984  7498 10701 23282]
3583 of 4196 written
[ 7393 36248 35300 19752  8426]
3584 of 4196 written
[34351 20985 13473  6446 35738]
3585 of 4196 written
[26736  7584 19637 31759  5578]
3586 of 4196 written
[30925  4605 17529  9730 29805]
3587 of 4196 written
[33398  8752  5945  8759  1447]
3588 of 4196 written
[   49 15687   876  6068 26024]
3589 of 4196 written
[19478 16367 22272 12029  8394]
3590 of 4196 written
[ 2623 29355  1397 29915 21495]
3591 of 4196 written
[28102  3366 30964 24488 27526]
3592 of 4196 w

[31177 27343  6314 11774 16169]
3771 of 4196 written
[   88 28750 11707 20941  8216]
3772 of 4196 written
[ 2686  2911 20204  9037 23188]
3773 of 4196 written
[31992 21406  9404 19642 20965]
3774 of 4196 written
[15593  7480 21512  2049 25830]
3775 of 4196 written
[14351 16016 21943 24362 30785]
3776 of 4196 written
[27132  4735 32083 29046 18483]
3777 of 4196 written
[12454 31992  1209 18830 29355]
3778 of 4196 written
[23050 13093 34715  9913 15469]
3779 of 4196 written
[ 7992 29805 35065 29219 10059]
3780 of 4196 written
[25876  6155  7967  8717 36213]
3781 of 4196 written
[29177  6111 17463 17382 27650]
3782 of 4196 written
[32265 22290 14644  6442 27923]
3783 of 4196 written
[33443 14701 26752  9577 15399]
3784 of 4196 written
[12077  2108 24639  1826 13797]
3785 of 4196 written
[24630  8049  2871 34664 15714]
3786 of 4196 written
[ 4800  5312 25823 15992  4327]
3787 of 4196 written
[10446  1076 16153 35738  2041]
3788 of 4196 written
[35300 29848 14616 26325 12354]
3789 of 4196 w

[25304  2339 33957  1289 17611]
3969 of 4196 written
[10645 16902 36634 33755 17240]
3970 of 4196 written
[18717 34713 11557  6637 16501]
3971 of 4196 written
[22002 14372 11977 28688  8519]
3972 of 4196 written
[18720 20407 10551 28200 33376]
3973 of 4196 written
[33818 13284  6473 10562 12077]
3974 of 4196 written
[18791 28102 25025 20407 13473]
3975 of 4196 written
[36872 36109  6595 22029 35300]
3976 of 4196 written
[27132  2339 31992 29046 19087]
3977 of 4196 written
[14785 21867 21812  1397  2133]
3978 of 4196 written
[34930 33562 12997 24488 21913]
3979 of 4196 written
[16153 15687 10446 24786  9164]
3980 of 4196 written
[33173 11366  2049 32479  2041]
3981 of 4196 written
[24479 27409    12 36613 21216]
3982 of 4196 written
[28308 14076 15469 30537  8013]
3983 of 4196 written
[35874 14249 16556 25920 14709]
3984 of 4196 written
[17205  9035 15080 37065  3958]
3985 of 4196 written
[30982 14094 30893  7567 13094]
3986 of 4196 written
[12354 21907 19072 16812   282]
3987 of 4196 w

[21913 17936  5578 11109  3223]
4170 of 4196 written
[22904  4327 31136  6193 21661]
4171 of 4196 written
[11458  7903  2148  9793 35319]
4172 of 4196 written
[35890 36241 13516 22920  4407]
4173 of 4196 written
[12305  9420  8252 36997 36855]
4174 of 4196 written
[16766  4039 10261 34517 30653]
4175 of 4196 written
[19127 25920 33266 22976  8216]
4176 of 4196 written
[28688  8875 22002 20620 23949]
4177 of 4196 written
[34602 16891 35513 31220 10642]
4178 of 4196 written
[23012  3665 12669  8235 19370]
4179 of 4196 written
[21279 26005 24098 30106 28485]
4180 of 4196 written
[ 3077 33443 24880 27361 15971]
4181 of 4196 written
[27526 30964 30925 25421 12669]
4182 of 4196 written
[23388 30755 35695  6699 24281]
4183 of 4196 written
[15687   876  3496  2578  6951]
4184 of 4196 written
[ 3006 23978  4197 16169 27275]
4185 of 4196 written
[ 7368 12873 22341 25694 12102]
4186 of 4196 written
[14174  7592 28083 26681 34348]
4187 of 4196 written
[24639 12997 25645 15971 24626]
4188 of 4196 w